# Dream Energy : Trafic Analysis around landmarks, based on Coyote databases

The goal of this notebook is to answer : What is the trafic situation around some landmarks ?

- Datasets :
    * The landmarks are provided by the client. They are composed of a name (=description), a latitude and a longitude.
    
    * The Coyote dataset. This dataset is present on the ALEIA server 172.16.201.164, in the database coyote.
        It contains the following information on the Coyote traces :
        - id of the vehicule
        - time in seconds since 1st second of 1970
        - GPS coordonates (= latitude, longitude)
        - speed of the vehicule
    
    * The journey datatet. This dataset is present on the ALEIA server 172.16.201.133, in the database coyote.
        A journey represents a squence of several Coyote traces. It contains the following information :
        - id of the vehicule
        - duration in seconds of the trip
        - distance in km of the trip
        - GPS coordonates (= latitude, longitude) of every Coyote points forming the trip
        
        
- Notebooks :
    * 1st part :
        Extract Coyote and Journey databases into csv format (useful for ALEIA plateform)
        Use of Pymongo librairie
        
    * 2nd part :
        For every landmark, count how many vehicules drived close to it.
        - Creation of a square around the landmark, based on his latitude, longitude
        - Compute distance landmark - Coyote trace, if the trace is in the square around the landmark
        
    * 3rd part :
        For every landmark, compute how many vehicules were traveling for more than a certain a distance (threshold(s) chosen by the client)
        - For every vehicules that went closed to a landmark, get his journey info in the journey database
        - Compute how many vehicules have a journey distance above the threshold.

### Import librairies

In [1]:
import pandas as pd
import numpy as np

import pymongo

import datetime
from tqdm import tqdm
import time
import glob
import os, shutil

from os import listdir
from os.path import isfile, join

import csv
import glob
from math import radians, cos, sin, asin, sqrt

import warnings
warnings.filterwarnings("ignore")

### Functions

In [2]:
def get_data_coyote(path_minutes,path_hours,col,start_date, end_date,period=60):    
    ''' function to extract data from MongoDB Coyote '''
    '''
    input :
    - col = database column
    - start_date (second since 1970) = time beginning of the extraction
    - end_date (second since 1970) = time end of the extraction
     period in second = batch extraction (several small requests rather than one big)
    
    output :
    - creation of Hours repository with csv for every two hours of the day   
    - Error if no folder Datasets with two folders Minutes and Hours already created
    
    '''
    
    #For every period, create a csv with the data
    for i in tqdm(range (start_date,end_date,period)):
        data_1 = [x['_id'] for x in col.find({'time': {'$gte': i,  '$lt': i+period}})]
        data_2 = [x['id'] for x in col.find({'time': {'$gte': i,  '$lt': i+period}})]
        data_3 = [x['time'] for x in col.find({'time': {'$gte': i,  '$lt': i+period}})]
        data_4 = [x['speed'] for x in col.find({'time': {'$gte': i,  '$lt': i+period}})]

        data_longitude = [x['loc']['coordinates'][0] for x in col.find({'time': {'$gte': i,  '$lt': i+period}})]
        data_latitude = [x['loc']['coordinates'][1] for x in col.find({'time': {'$gte': i,  '$lt': i+period}})]

        data_tuples = list(zip(data_1,data_2,data_3,data_4,data_latitude,data_longitude))
        df = pd.DataFrame(data_tuples, columns=['_id','id_cars','time','speed','latitude','longitude'])
        df.to_csv(path_minutes + '/' + str(i) + '.csv')
        
    #Concat every small csv created before in one big (due to the period parameter)
    all_files = glob.glob(path_minutes + "/*.csv")
    li = []
    for filename in tqdm(all_files):
        df = pd.read_csv(filename, index_col=None, header=0, encoding = 'cp1252')
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    frame.to_csv(path_hours + '/' + str(i) + '.csv')

    #Annexe : CLean folder with the small csv files.
    for filename in os.listdir(path_minutes):
        file_path = os.path.join(path_minutes, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [3]:
def dossier_stockage_resultat(noms_points,parent_dir):
    '''
    Function to create the folder where the results of phase 1 will be saved, before phase 2
    input :
    - noms_points = list
    - parents_dir = folder where to save the ID of the vehicule
    
    output
    - create new folders
    
    '''
    
    
    for nom in noms_points:
        path = os.path.join(parent_dir, nom) 
        if not os.path.exists(path):
            os.mkdir(path)
            
def concat_csv(path,extension):
    '''
    Function to concat csv files
    
    input : 
    - path = path of the csv files to concatenate in one csv
    - extension (string) = filter on the end of the name of csv files to concat
    
    output :
    - the dataframe concatenated    
    
    '''
    
    all_files = glob.glob(path + "/*" + extension)
    li = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0, encoding = 'cp1252')
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    return frame

In [4]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points on the earth (specified in decimal degrees)
    
    input :
    - lon1, lat1 = decimal degrees cordinates of first points
    - lon2, lat2 = decimal degrees cordinates of second points
    
    output :
    -   the distance between the two points
    
    """
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6372.8 # Radius of earth in kilometers. 
    
    return c * r

In [5]:
def comparaison_distance_methode_carre(latitude_ref,longitude_ref,latitude_car,longitude_car,radius):
    '''
    Function to check if a car is in a given radius around the landmark

    input :
    - latitude_ref, longitude_ref = ref point coordinates
    - latitude, longitude = cars gps coordinates
    - radius = limit radius of cars counting in km
    
    return
     - 1 if the car is in the radius
     - 0 otherwise  
    
    '''
    
    if (haversine(longitude_car, latitude_car, longitude_ref, latitude_ref) <= radius):
        return 1
    else:
        return 0

In [6]:
def compute_distance_trip(trip):
    '''
    TODO
    
    '''
    
    distance_km = 0
    
    for i in range(1,len(trip)):
        
        longitude_1 = trip[i-1]['coordinates'][0]
        longitude_2 = trip[i]['coordinates'][0]
        latitude_1 = trip[i-1]['coordinates'][1]    
        latitude_2 = trip[i]['coordinates'][1]
    
        distance_km = distance_km + haversine(longitude_1, latitude_1, longitude_2, latitude_2)
        
    return round(distance_km,3)

In [7]:
def add_column_counter(source, radius, threshold, isabove = True):
    '''
    Function to compute how many cars, that drove within a certain radius around the landmark, travelled for a distance above a threshold (= compute long trips)
    
    input :
    - source : folder with distance csv files for every landmark
    - radius : radius around the landmark
    - threshold : distance threshold of the trip in km
    
    output : 
    - two columns dataframe :
        - landmarks name/description
        - number of vehicule with a travelled distance superior to the threshold     
    
    '''
    count = []
    name = []
    all_files = glob.glob(source + "/*" + str(radius) + "km*")
    
    for filename in all_files:
        dataframe = pd.read_csv(filename, encoding = 'cp1252')
        name.append(filename[75:-4])
        if (isabove):
            count.append(dataframe[dataframe.distance >= threshold].shape[0])
            title = 'nombre_vehicule_' + str(radius) + 'km_distance_sup_seuil_' + str(threshold) + 'km'
        else:
            count.append(dataframe[dataframe.distance < threshold].shape[0])
            title = 'nombre_vehicule_' + str(radius) + 'km_distance_inf_seuil_' + str(threshold) + 'km'

    label = np.array(count)  
    
    dataset = pd.DataFrame({'description': name, title : label}, columns = ['description', title])
    return dataset

In [8]:
#Phase 1 : Count cars near the landmarks

def nombre_vehicule_proche(dataframe, dataframe_points, titre_csv, km_autour_du_point, path):
    
    
    latitude_points = np.array(dataframe_points.coord_1)
    longitude_points = np.array(dataframe_points.coord_2)
    noms_points = np.array(dataframe_points.description) # A modifier selon le cas
    
    N = dataframe_points.shape[0]
    dossier_stockage_resultat(noms_points,path)
    
    latitude = np.array(dataframe.latitude)
    longitude = np.array(dataframe.longitude)
    
    tab_inter_latitude = np.tile(np.array(latitude), (N, 1))
    tab_inter_longitude = np.tile(np.array(longitude), (N, 1))

    for j in km_autour_du_point:
        
        cond_1 = tab_inter_latitude[np.absolute(tab_inter_latitude - latitude_points.reshape(N,1)) <= j/111]
        cond_2 = tab_inter_longitude[(np.absolute(tab_inter_longitude - longitude_points.reshape(N,1))) <= j/85]
        dataframe_int = dataframe[dataframe.latitude.isin(cond_1) & dataframe.longitude.isin(cond_2)]
                              
        for i in range(N):
            latitude_int = np.array(dataframe_int.latitude)
            longitude_int = np.array(dataframe_int.longitude)
    
            cond_3 = latitude_int[(np.absolute(latitude_int - latitude_points[i])) <= j/111]
            cond_4 = longitude_int[(np.absolute(longitude_int - longitude_points[i])) <= j/85]
            dataframe_int_2 = dataframe_int[dataframe_int.latitude.isin(cond_3) & dataframe_int.longitude.isin(cond_4)]
            dataframe_int_2['isclose'] = 0

            dataframe_int_2['isclose'] = dataframe_int_2.apply(lambda x: comparaison_distance_methode_carre(latitude_points[i],longitude_points[i],x['latitude'],x['longitude'],j),axis=1)
            dataframe_int_3 = dataframe_int_2[dataframe_int_2['isclose'] == 1][['_id','time','id_cars','latitude','longitude']]
            
            dataframe_int_3.groupby('id_cars').first().to_csv(path + '/'  + str(noms_points[i]) + '/' + titre_csv + '_' + str(j) + 'km.csv', encoding = 'cp1252')

#Phase 2 : Compute cars trips distance for every vehicules close to landmarks

def distance_vehicule_trajet(source,col_trajets,titre,path_distance):
    dataframe = pd.read_csv(source, encoding = 'cp1252')

    taille = dataframe.shape[0]
    dataframe['distance'] = np.zeros(taille)
    
    data = [x for x in col_trajets.find({'id': {'$in': dataframe.id_cars.to_list()}})]
    df = pd.DataFrame(data)         
           
    if (df.shape[0] == 0):
        return 0
    
    else:
        
        for identifiant in dataframe['id_cars']: #Pour dataframe 
            df_inter = df[df['id'] == identifiant].reset_index()
            N = df_inter.shape[0]
            if (N == 0):
                dataframe.loc[dataframe.id_cars == identifiant, 'distance'] = 0
            elif (N == 1):
                dataframe.loc[dataframe.id_cars == identifiant, 'distance'] = df_inter['trip_distance_km'][0]
            else :
                for j in range(N): #Pour les trajets présents 
                    distance_off = 0
                    if ((df_inter['begin'][j]['time'] <  dataframe.loc[dataframe.id_cars == identifiant]['time'].iloc[0]) & (dataframe.loc[dataframe.id_cars == identifiant]['time'].iloc[0] - 3*3600 < df_inter['begin'][j]['time'])):
                        distance_off = df_inter['trip_distance_km'][j]
                        dataframe.loc[dataframe.id_cars == identifiant, 'distance'] = distance_off
                        break
                
        n_vehicule_trajet = dataframe[dataframe['distance'] > 0]
    
        n_vehicule_sans_trajet = dataframe[dataframe['distance'] == 0].shape[0]
    
        n_vehicule_trajet.to_csv(path_distance + '/' + titre + '.csv', encoding = 'cp1252')
        return n_vehicule_sans_trajet

In [9]:
def distance_vehicule_trajet(source,col_trajets,titre,path_distance):
    dataframe = pd.read_csv(source, encoding = 'cp1252')

    taille = dataframe.shape[0]
    dataframe['distance'] = np.zeros(taille)

    data = [x for x in col_trajets.find({'id': {'$in': dataframe.id_cars.to_list()}})]
    df = pd.DataFrame(data) 
    
    for identifiant in dataframe['id_cars']: 
        
        #Creer un dataframe intermediaire avec uniquement les trajets de ce véhicule
        df_inter = df[df['id'] == identifiant].reset_index()
        N = df_inter.shape[0]
        
        #Si le véhicule possède 1 ou plusieurs trajets :
        if (N != 0):
            for j in range(N):
            
                #On vérifie que t(entré véhicule dans zone proche) est inf à t(fin de trajet) et sup à t(début de trajet)
                if ((df_inter['begin'][j]['time'] <  dataframe.loc[dataframe.id_cars == identifiant]['time'].iloc[0]) & (df_inter['end'][j]['time'] > dataframe.loc[dataframe.id_cars == identifiant]['time'].iloc[0])):
                
                #Si plusieurs trajets, prendre celui ou il y a le points GPS d'entrée de zone
                    trips_points = df_inter['loc'].iloc[j]
                    latitude_point_zone = dataframe.loc[dataframe.id_cars == identifiant]['latitude'].iloc[0]
                    longitude_point_zone = dataframe.loc[dataframe.id_cars == identifiant]['longitude'].iloc[0]
               
                    for k in range(len(trips_points)):
                        if (trips_points[k]['coordinates'] == [longitude_point_zone, latitude_point_zone]):                      
                            trip = trips_points[:k+1]
                            dataframe.loc[dataframe.id_cars == identifiant, 'distance'] = compute_distance_trip(trip)
                            break
                    break
                                                
                #Si le points GPS d'entrée de zone apparait plusieurs fois, prendre celui dont t est le plus faible (1er point d'entrer)
                #Si aucun trajet avec le points GPS d'entrée de zone, retourner 0 en distance
                
                
    n_vehicule_trajet = dataframe[dataframe['distance'] > 0]
    n_vehicule_trajet.to_csv(path_distance + '/' + titre + '.csv', encoding = 'cp1252')
        
    n_vehicule_sans_trajet = dataframe[dataframe['distance'] == 0].shape[0]        
    return round(n_vehicule_sans_trajet / dataframe.shape[0] * 100,1)

In [10]:
#Phase 3 : Creation of the final csv with phase 1 and 2 results

def compte_rendu_csv(source_distance,path_result,file,threshold_1,threshold_2,radius):
    
    data_rendu = pd.read_csv(path_result + '/' + file + '/carre_rendu_final.csv', encoding = 'cp1252')
    
    for j in radius:
        data_trajets_manquants = pd.read_csv(path_result + '/' + file + '/n_vehicule_sans_trajets_' + str(j) + 'km.csv', encoding = 'cp1252')

        dataset_0 = add_column_counter(source_distance,j,threshold_1, isabove = False)
        data_rendu = data_rendu.merge(dataset_0, how='left', on='description')
        
        dataset_1 = add_column_counter(source_distance,j,threshold_1)
        data_rendu = data_rendu.merge(dataset_1, how='left', on='description')
        
        dataset_2 = add_column_counter(source_distance,j,threshold_2)
        data_rendu = data_rendu.merge(dataset_2, how='left', on='description')
        
        data_rendu['nombre_vehicule_' + str(j) + 'km_distance_seuil_' + str(threshold_1) + '_' + str(threshold_2) + 'km'] = data_rendu['nombre_vehicule_' + str(j) + 'km_distance_sup_seuil_' + str(threshold_1) + 'km'] - data_rendu['nombre_vehicule_' + str(j) + 'km_distance_sup_seuil_' + str(threshold_2) + 'km']
        data_rendu = data_rendu.drop(['nombre_vehicule_' + str(j) + 'km_distance_sup_seuil_' + str(threshold_1) + 'km'], axis=1)
        
        data_rendu = data_rendu.merge(data_trajets_manquants[['description','pourcentage_vehicules_trajets_manquants_' + str(j) + 'km']], how='left', on='description')
        
    return data_rendu

In [11]:
def main(source_client, source_gps, path_rendu, path_inter, path_distance, km_autour_du_point, col_trajets, name,seuil_a,seuil_b):
    
    #Remove each file in the ID_cars folder. Folder has to be empty for next steps to proceed
    folderlist = glob.glob(os.path.join(path_inter, "*"))
    for f in folderlist:
        filelist = glob.glob(os.path.join(f, "*"))
        for file in filelist:
            os.remove(file)
        os.rmdir(f)

    #Import data (ref)
    dataframe_client = pd.read_csv(source_client, encoding = 'cp1252')
    #List all files in the gps folder (here : the 2 hours csv files with the data from the coyote database)
    onlyfiles = [f for f in os.listdir(source_gps) if os.path.isfile(os.path.join(source_gps, f))]
    
    
    #Phase 1 (1st part)
    start_time = time.time()
    for file in tqdm(onlyfiles): # CSV gps (2H)
        dataframe = pd.read_csv(source_gps + file, encoding = 'cp1252')
        nombre_vehicule_proche(dataframe,dataframe_client,str(file[:10]),km_autour_du_point,path_inter)
        
    print("--- Compteur Véhicule par heure : %s seconds ---" % (time.time() - start_time))
    
    #Final Dataframe creation
    dataframe_final = dataframe_client
    
    directory_contents = os.listdir(path_inter)

    #Phase 1 (2nd part)
    for j in km_autour_du_point: #2 rayon de reparage
        dataframe_final['n_vehicules_rayon_' + str(j) +'km'] = np.zeros(dataframe_final.shape[0])
        for k in directory_contents:
            dataframe_km = concat_csv(path_inter + '/' + k, str(j) + 'km.csv')
            dataframe_final.loc[dataframe_final.description == k, 'n_vehicules_rayon_' + str(j) +'km'] = dataframe_km['id_cars'].unique().size
            dataframe_km.groupby('id_cars').first().to_csv(path_inter + '/'  + str(k) + '/' + 'all_day_' + str(j) + 'km.csv', encoding = 'cp1252')
                

    dataframe_final.to_csv(path_rendu + '/' + str(name) + '/carre_rendu_final.csv', index = False, encoding = 'cp1252')
    
    #Remove each file in the Distance folder. Folder has to be empty for next steps to proceed
    filelist = glob.glob(os.path.join(path_distance, "*"))
    for f in filelist:
        os.remove(f)

    #Phase 2    
    start_time = time.time()
    for j in km_autour_du_point:
        prop_vehicules_sans_trajets = []
        for i in tqdm(directory_contents):
            path_km = path_inter + '/' + i + '/all_day_' + str(j) + 'km.csv'
            n_vehicules = distance_vehicule_trajet(path_km, col_trajets,'client_' + str(j) + 'km_' + i,path_distance)
            prop_vehicules_sans_trajets.append(n_vehicules)
            
        pd.DataFrame({'description': directory_contents, 'pourcentage_vehicules_trajets_manquants_' + str(j) + 'km' : prop_vehicules_sans_trajets}, columns = ['description', 'pourcentage_vehicules_trajets_manquants_' + str(j) + 'km']).to_csv(path_rendu + '/' + str(name) + '/n_vehicule_sans_trajets_' + str(j) + 'km.csv', encoding = 'cp1252')
        
    print("--- Distribution trajet : %s seconds ---" % (time.time() - start_time))
    
    #Phase 3
    
    data_rendu_final = compte_rendu_csv(path_distance,path_rendu,name,seuil_a,seuil_b,km_autour_du_point)
    data_rendu_final.to_csv(path_rendu + '/' + str(name) + '/rendu_final.csv', encoding = 'cp1252')
    

### Main

In [12]:
myclient = pymongo.MongoClient("mongodb://172.16.201.164:27017/")
mydb = myclient["coyote"]
mycol = mydb["coyote_data_20200113"]

myclient_trajets = pymongo.MongoClient("mongodb://172.16.201.133:27017/")
mydb_trajets = myclient_trajets["coyote"]
mycol_trajets = mydb_trajets["trajet_20200113"]

path_gps = 'C:/Users/NicolasSTUCKI/Documents/Dream Energy/Datasets/Hours_plateforme_format/'
path_minutes = 'C:/Users/NicolasSTUCKI/Documents/Dream Energy/Datasets/Minutes/'

start = datetime.datetime(2020,1,14,0).timestamp()
end = datetime.datetime(2020,1,15,0).timestamp()

In [13]:
#for i in range(int(start),int(end),7200):
    #get_data_coyote(path_minutes,path_gps,mycol,i,i+7200)

In [14]:
path_repere = 'C:/Users/NicolasSTUCKI/Documents/Dream Energy/Datasets/Repere'
path_rendu = 'C:/Users/NicolasSTUCKI/Documents/Dream Energy/Datasets/Rendu'
path_inter = 'C:/Users/NicolasSTUCKI/Documents/Dream Energy/Datasets/Id_voiture'
path_distance = 'C:/Users/NicolasSTUCKI/Documents/Dream Energy/Datasets/Distance'

radius_list = [3,5]
seuil_1 = 40
seuil_2 = 150

filesrepere = [f[:-4] for f in listdir(path_repere) if isfile(join(path_repere, f))]
dossier_stockage_resultat(filesrepere,path_rendu)

In [ ]:
for filerepere in tqdm(filesrepere):
    repere = path_repere + '/' + str(filerepere) + '.csv'
    main(repere, path_gps, path_rendu, path_inter, path_distance, radius_list, mycol_trajets,filerepere,seuil_1,seuil_2)

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [04:31<00:00, 14.28s/it]


--- Compteur Véhicule par heure : 271.4006609916687 seconds ---



100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [02:45<00:00,  9.21s/it]

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [03:49<00:00, 12.76s/it]


--- Distribution trajet : 395.45874977111816 seconds ---


 42%|██████████████████████████████████▉                                                | 8/19 [01:27<02:51, 15.63s/it]

In [ ]:
#pd.read_csv('./Datasets/Repere/Terrains_DE_cleaned_p2.csv', sep = ',', encoding = 'cp1252')